In [1]:
import os
%cd ..

d:\project_task_extra\deep learning\Chicken-Disease-Classification


In [2]:
import tensorflow as tf
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [3]:
from dataclasses import dataclass
from pathlib import Path
from typing import List
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params : dict
    params_image_size: List[int]
    params_batch_size: int

In [4]:
from Chicken_Disease_classification.constants import *
from Chicken_Disease_classification.utils.common import create_directories,save_json,save_bin,read_yaml

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        
        
        
    def get_validation_config(self)->EvaluationConfig:
        
        evaluation_config =EvaluationConfig(
            path_of_model='artifacts/training/model.h5',
            training_data='artifacts/data_ingestion/Chicken-fecal-images',
            all_params=self.params,
            params_batch_size=self.params.BATCH_SIZE,
            params_image_size=self.params.IMAGE_SIZE
            )
        
        return evaluation_config

In [6]:
from urllib.parse import urlparse
from datetime import datetime


In [9]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    @staticmethod
    def get_current_time_stamp():
        return f"{datetime.now().strftime('%Y%m%d%H')}"
    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        print(self.score)

    
    def save_score(self):
        scores = {f"loss_{Evaluation.get_current_time_stamp()}": self.score[0], f"accuracy_{Evaluation.get_current_time_stamp()}": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)


In [10]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e

[2023-06-08 16:58:37,295] || common.py || 32 || root || read_yaml() || 32 || INFO || yaml file : config\config.yaml loaded successfully
[2023-06-08 16:58:37,298] || common.py || 32 || root || read_yaml() || 32 || INFO || yaml file : params.yaml loaded successfully
[2023-06-08 16:58:37,299] || common.py || 67 || root || create_directories() || 67 || INFO || Directory artifacts already exists
Found 116 images belonging to 2 classes.
8/8 [==============================] - 7s 888ms/step - loss: 0.5348 - accuracy: 0.9310
[0.5347861051559448, 0.931034505367279]
[2023-06-08 16:58:45,039] || common.py || 102 || root || save_json() || 102 || INFO || JSON file updated at scores.json
